This is going to be a survey of various techniques implemented for the purpose of image classification of two classes. In this case we are looking at "Bunny" versus "Not Bunny". "Not Bunny" can we a wide category by which implementing solutions given an unbalanced dataset can be harder to understand best practices.

Variables taken into consideration:

<ol>
    <li>Amount of Data</li>
    <li>Data Partitioning</li>
    <ol>
        <li>50% Bunny and 50% Not Bunny</li>
        <li>10% Bunny, and 10% of 9 other classes.</li>
    </ol>
    <li>Testing Layer</li>
    <ol>
        <li>out_relu</li>
        <li>Conv_1_bn</li>
        <li>block_16_project_BN</li>
        <li>block_16_project</li>
        <li>block_16_depthwise_relu</li>
        <li>block_16_expand</li>
        <li>block_15_add</li>
    </ol>
    <li>Data Augmentation</li>
    <ol>
        <li>None (Reshape if needed)</li>
        <li>Averaging</li>
        <li>UMAP</li>
    </ol>
    <li>Prediction Technique</li>
    <ol>
        <li>SVM (https://ieeexplore.ieee.org/document/8623118)</li>
        <li>NN Layer</li>
        <li>KNN</li>
    </ol>
    <li>Technique Hyperparameters</li>
    <ol>
        <li>Kernel Type (SVM)</li>
        <li>Dropout Rate (NN)</li>
    </ol>
</ol>

The following are all imports required to run the code below.

In [1]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from keras.models import Model

The following are all functions used for the purpose of creating datasets.

In [3]:
# Directory extensions used for class creations
class_dirs = ["/Human", "/Nature-Background", "/Text", "/Dogs", "/Cats", "/Hamsters", "/Suggestively-Sexual",
              "/Suggestively-Violent", "/Abstract-Background", "/Empty-Cages", "/Bunny-Drawings"]

# Deleting and recreating directories depending on partition
def refresh_dirs(path, partition):
    # Removing extra directories
    class_dirs_plus = class_dirs.append("/Other-Usable")
    for name in class_dirs:  
        try:
            shutil.rmtree(str(path) + "/Images-Using" + str(name))
        except Exception as e:
            print("Couldn't remove " + str(path) + "/Images-Using" + str(name))
    
    # Making directories depending on what partitions are desired
    if partition == "50/50":
        os.mkdir(path + "/Images-Using/Other-Usable")
    elif partition == "Evenly Split":
        for name in class_dirs:
            os.mkdir(str(path) + "/Images-Using" + str(name))

# Creating images and over-sampling if need be
def do_splits(path, partition):
    if partition == "50/50":
        for class_idx in class_dirs:
            allFileNames = os.listdir(path + partition)
            allFileNames = [path + partition + '/' + name for name in allFileNames]
            other_directory = "/Images-Using/Other-Usable"

            idx = 0
            for name in allFileNames:
                failed_naming = True
                while failed_naming:
                    try:
                        shutil.copy(name, path + other_directory)
                        os.rename(path + other_directory + '/' + name.rsplit('/', 1)[-1],
                                  path + other_directory + '/' + str(class_idx) + '-' + str(idx) + name.rsplit('/', 1)[-1])
                        failed_naming = False
                    except:
                        idx += 1
                    idx += 1
    elif partition == "Evenly Split":
        # todo:
        #
        #
        #
        #
        #
        #
        #
        #
        #
        #
        #
        #
        #
        #
        #
        #

                    
def grab_datasets(path):
    # Size set for mobilenetv2
    batch_size = 1
    img_height = 160
    img_width = 160
    
    # Training, validation and test sets created
    train_ds = tf.keras.utils.image_dataset_from_directory(
        path + "/Images-Using",
        validation_split=0.2,
        subset="training",
        seed=321,
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    validation_ds = tf.keras.utils.image_dataset_from_directory(
        path + "/Images-Using",
        validation_split=0.2,
        subset="validation",
        seed=321,
        image_size=(img_height, img_width),
        batch_size=batch_size
    )
    
    val_batches = tf.data.experimental.cardinality(validation_ds)
    test_ds = validation_ds.take(val_batches // 4)
    validation_ds = validation_ds.skip(val_batches // 4)

    # Setting up speedy image fetching to avoid bottlenecking
    AUTOTUNE = tf.data.AUTOTUNE

    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    validation_ds = validation_ds.prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

    return train_ds, validation_ds, test_ds

SyntaxError: invalid syntax (138375254.py, line 42)

In [2]:
data_percentages = [100]
data_partitions = ["50/50","Evenly Split"]
testing_layers = ["out_relu", "Conv_1_bn"]
data_augmentations = ["none", "averaging", "UMAP"]
prediction_techniques = ["SVM", "NN", "KNN"]

for percentage in data_percentages:
    for partition in data_partitions:
        
        # Set path to file code is being ran at
        images_path = os.getcwd()
        # Delete files from previous experiment
        refresh_dirs(images_path, partition)
        # Create splits of what is considered training/testing data and how many classes are recognized
        do_splits(images_path, partition)
        # Grab Keras datasets
        train_ds, validation_ds, test_ds = grab_datasets(images_path)
        
        batch_size = 1
        img_height = 160
        img_width = 160
        
        
        
        for layer in testing_layers:
            for augmentation in data_augmentations:
                for technique in prediction_techniques:
                    